# Load Ni-Mo data

In [1]:
from pymatgen import Structure
from monty.serialization import loadfn

data = loadfn('data.json')
train_structures = [d['structure'] for d in data]
train_energies = [d['outputs']['energy'] for d in data]
train_forces = [d['outputs']['forces'] for d in data]

vasp_stress_order = ['xx', 'yy', 'zz', 'xy', 'yz', 'xz']
snap_stress_order = ['xx', 'yy', 'zz', 'yz', 'xz', 'xy']
train_stresses = []
for d in data:
    virial_stress = d['outputs']['stress']
    train_stresses.append([virial_stress[vasp_stress_order.index(n)] * 0.1 for n in snap_stress_order]) # convert kbar to GPa

# Setup the initial weights for training (If not, the weights for energy, force, and stress will be both equal to 1)

In [2]:
import numpy as np
from maml.utils import pool_from, convert_docs

train_pool = pool_from(train_structures, train_energies, train_forces, train_stresses)
_, df = convert_docs(train_pool, include_stress=True)

weights = np.ones(len(df['dtype']), )

# set the weights for energy equal to 100
weights[df['dtype'] == 'energy'] = 100
weights[df['dtype'] == 'force'] = 1
weights[df['dtype'] == 'stress'] = 0.01

# Set up the SNAP and train

In [3]:
from maml.base import SKLModel
from maml.describers import BispectrumCoefficients
from sklearn.linear_model import LinearRegression
from maml.apps.pes import SNAPotential

element_profile = {'Mo': {'r': 5.0, 'w': 1}, 'Ni': {'r': 5.0, 'w': 1}}
describer = BispectrumCoefficients(rcutfac=0.5, twojmax=6, element_profile=element_profile, 
                                   quadratic=False, pot_fit=True, include_stress=True)
model = SKLModel(describer=describer, model=LinearRegression())
snap = SNAPotential(model=model)
snap.train(train_structures, train_energies, train_forces, train_stresses, include_stress=True, sample_weight=weights)

INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_serial
INFO:maml.utils._lammps:Structure index 0 is rotated.
INFO:maml.utils._lammps:Structure index 1 is rotated.
INFO:maml.utils._lammps:Structure index 2 is rotated.
INFO:maml.utils._lammps:Structure index 3 is rotated.
INFO:maml.utils._lammps:Structure index 4 is rotated.
INFO:maml.utils._lammps:Structure index 5 is rotated.
INFO:maml.utils._lammps:Structure index 6 is rotated.
INFO:maml.utils._lammps:Structure index 7 is rotated.
INFO:maml.utils._lammps:Structure index 8 is rotated.
INFO:maml.utils._lammps:Structure index 9 is rotated.


# Predict the energies, forces, and stresses of training data

In [4]:
df_orig, df_predict = snap.evaluate(test_structures=train_structures, 
                                    test_energies=train_energies,
                                    test_forces=train_forces, 
                                    test_stresses=train_stresses,
                                    include_stress=True)

INFO:maml.utils._lammps:Structure index 0 is rotated.
INFO:maml.utils._lammps:Structure index 1 is rotated.
INFO:maml.utils._lammps:Structure index 2 is rotated.
INFO:maml.utils._lammps:Structure index 3 is rotated.
INFO:maml.utils._lammps:Structure index 4 is rotated.
INFO:maml.utils._lammps:Structure index 5 is rotated.
INFO:maml.utils._lammps:Structure index 6 is rotated.
INFO:maml.utils._lammps:Structure index 7 is rotated.
INFO:maml.utils._lammps:Structure index 8 is rotated.
INFO:maml.utils._lammps:Structure index 9 is rotated.


In [5]:
df_predict[df_predict['dtype'] == 'energy']

,y_orig,n,dtype
0,-1022.306395,144.0,energy
439,-1021.230551,144.0,energy
878,-968.144050,144.0,energy
1317,-974.611306,144.0,energy
1756,-968.823465,144.0,energy
2195,-1008.715833,144.0,energy
2634,-1020.500077,144.0,energy
3073,-958.943061,144.0,energy
3512,-1008.551143,144.0,energy
3951,-972.534965,144.0,energy


# Lattice constants, Elastic constant
### Large error due to limited training data -- 10 structures

In [ ]:
from pymatgen.core import Lattice

Ni = Structure.from_spacegroup(sg='Fm-3m', species=['Ni'], lattice=Lattice.cubic(3.51), coords=[[0, 0, 0]])
Mo = Structure.from_spacegroup(sg='Im-3m', species=['Mo'], lattice=Lattice.cubic(3.17), coords=[[0, 0, 0]])

In [8]:
from maml.apps.pes import LatticeConstant

lc_calculator = LatticeConstant(ff_settings=snap)
a, b, c = lc_calculator.calculate([Ni])[0]
print('Ni', 'Lattice a: {}, Lattice b: {}, Lattice c: {}'.format(a, b, c))

INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_serial


Ni Lattice a: 3.59493228412543, Lattice b: 3.59493228412543, Lattice c: 3.59493228412543


In [9]:
lc_calculator = LatticeConstant(ff_settings=snap)
a, b, c = lc_calculator.calculate([Mo])[0]
print('Mo', 'Lattice a: {}, Lattice b: {}, Lattice c: {}'.format(a, b, c))

INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_serial


Mo Lattice a: 3.05442684929677, Lattice b: 3.05442684929677, Lattice c: 3.05442684929678


In [6]:
from maml.apps.pes import ElasticConstant

Ni_ec_calculator = ElasticConstant(ff_settings=snap)
Ni_C11, Ni_C12, Ni_C44, _ = Ni_ec_calculator.calculate()
print('Ni', ' C11: ', Ni_C11, 'C12: ', Ni_C12, 'C44: ', Ni_C44)

INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_serial


Ni  C11:  314.169856583909 C12:  308.660699662111 C44:  69.7476168000187


In [7]:
Mo_ec_calculator = ElasticConstant(ff_settings=snap)
Mo_C11, Mo_C12, Mo_C44, _ = Mo_ec_calculator.calculate()
print('Mo', ' C11: ', Mo_C11, 'C12: ', Mo_C12, 'C44: ', Mo_C44)

INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_serial


Mo  C11:  33.0332580471668 C12:  129.122105036582 C44:  4.45521235633949
